In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import urllib.parse

# Database credentials and details
# host = "localhost"
# port = 3306
# user = "root"
# password = "1245"
# database = "firewall"

host="localhost"
port=3306
user="root"
password="root@123"
database="firewall"
encoded_password = urllib.parse.quote_plus(password)

## Parser is needed since password contain @


db_uri = f"mysql+pymysql://{user}:{encoded_password}@{host}:{port}/{database}"
engine = create_engine(db_uri)

# SQLAlchemy connection string (URI)
# db_uri = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"

# Create the SQLAlchemy engine
# engine = create_engine(db_uri)
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer(
#         'sentence-transformers/all-MiniLM-L6-v2', device='cpu:0')

def make_operator_df():

    with engine.connect() as conn:

        tbl_operator_rule_query = """

            SELECT *
            FROM tbl_operator_rule
            WHERE rule IN (SELECT id FROM tbl_rules_config WHERE rule LIKE 'ai') 
            


        """

        tbl_operator_rule_df = pd.read_sql_query(tbl_operator_rule_query, conn)
        tbl_global_rule_query = """

        SELECT * FROM firewall.tbl_rule WHERE rule IN (SELECT id FROM tbl_rules_config WHERE rule LIKE 'ai');
            
        """

        tbl_global_rule_df = pd.read_sql_query(tbl_global_rule_query, conn)
        operator_rule_settings_query = """
        SELECT * 
        FROM `tbl_rule_setting` 
        WHERE `rule` IN (
            SELECT id
            FROM tbl_operator_rule
            WHERE rule IN (SELECT id FROM tbl_rules_config WHERE rule LIKE 'ai') 
            
            
        )
        AND `tbl` = 'tbl_operator_rule';
        """

        # Execute the query and load into DataFrame

        operator_rule_settings_df = pd.read_sql_query(
            operator_rule_settings_query, conn)

        global_rule_settings_query = """
        SELECT *  FROM `tbl_rule_setting` WHERE `rule` in (SELECT id FROM firewall.tbl_rule WHERE rule IN (SELECT id FROM tbl_rules_config WHERE rule LIKE 'ai'))AND tbl = 'tbl_rule'
        """

        # Execute the query and load into DataFrame

        global_rule_settings_df = pd.read_sql_query(
            global_rule_settings_query, conn)

        tbl_operator_rule_df['global_bypass'] = tbl_global_rule_df['bypass'].iloc[0]
        tbl_operator_rule_df.rename(
            columns={'bypass': 'operator_bypass'}, inplace=True)

        def determine_rule_owner(row):
            if row['operator_bypass'] == 0:
                return 'operator'
            elif row['operator_bypass'] == 1 and row['global_bypass'] == 0:
                return 'global'
            else:
                return 'none'
        tbl_operator_rule_df['rule_owner'] = tbl_operator_rule_df.apply(
            determine_rule_owner, axis=1)

        filtered_operator_df = tbl_operator_rule_df[tbl_operator_rule_df['rule_owner'] != 'none']

        # Initialize a list to store the new rows
        new_rows = []

        # Iterate over the filtered rows
        for idx, row in filtered_operator_df.iterrows():
            operator = row['operator']
            rule_id = row['id']
            rule_owner = row['rule_owner']

            if rule_owner == 'operator':
                # Get settings from operator_rule_settings_df
                settings_df = operator_rule_settings_df[operator_rule_settings_df['rule'] == rule_id]
            elif rule_owner == 'global':
                # Use global_rule_settings_df
                settings_df = global_rule_settings_df

            # Extract required values
            auto_train = settings_df[settings_df['name']
                                     == 'auto_train']['value'].values[0]
            detect = settings_df[settings_df['name']
                                 == 'detect']['value'].values[0]
            detect_score = settings_df[settings_df['name']
                                       == 'detect_score']['value'].values[0]
            manual_train = settings_df[settings_df['name']
                                       == 'manual_train']['value'].values[0]

            # Append to the new_rows list
            new_rows.append({'operator_name': operator, 'auto_train': auto_train,
                            'manual_train': manual_train, 'detect': detect, 'block_score': detect_score, })

        # Create new DataFrame

        default_row = {
            'operator_name': [10000],
            'auto_train': [0],
            'manual_train': [1],
            'detect': [0],
            'block_score': [99]
        }

        operator_df = pd.DataFrame(default_row)
        new_rows_df = pd.DataFrame(new_rows)
        operator_df = pd.concat([operator_df, new_rows_df], ignore_index=True)
        operator_df['operator_name'] = operator_df['operator_name'].astype(int)
        operator_df['auto_train'] = operator_df['auto_train'].astype(int)
        operator_df['manual_train'] = operator_df['manual_train'].astype(int)
        operator_df['detect'] = operator_df['detect'].astype(int)

        # Convert the last column to float
        operator_df['block_score'] = operator_df['block_score'].astype(float)
        operator_df['block_score'] = operator_df['block_score']/100

    return operator_df

operator_df = make_operator_df()

In [3]:
operator_df

,operator_name,auto_train,manual_train,detect,block_score
0,10000,0,1,0,0.99
1,139,1,1,0,0.50
